In [1]:
import os
import dotenv

dotenv.load_dotenv(override=True)

True

In [2]:
from langchain.chat_models import init_chat_model

model_name = 'llama-3.3-70b-versatile'
llm = init_chat_model(model_name, model_provider="groq")

In [3]:
from langchain.agents import tool
from langgraph.prebuilt import create_react_agent

profile_saved = False

@tool
def save_profile(data: dict) -> str:
    """Сохраняет профиль пользователя.
    Профиль пользователя состоит из следующих полей:
    - name: Имя
    - age: Возраст - целое число больше нуля. Если пользователь ввёл строку или не число, то это неверный ответ. Например, 2.5 или "-2" - это не возраст.
    - genre: Любимый жанр книг
    - color:Любимый цвет
    
    Возвращает сообщение об успешном сохранении профиля.
    
    
    
    """
    global profile_saved
    print("\n✅ Все данные собраны:")
    for k, v in data.items():
        print(f"- {k}: {v}")
    profile_saved = True
    return "Профиль успешно сохранён."


tools = [save_profile]

agent = create_react_agent(
    model=llm,
    tools=tools,
    prompt='''Ты - дружелюбный помощник, который поэтапно собирает информацию от пользователя.
Твоя задача: получить следующие данные:
- Имя
- Возраст
- Любимый жанр книг
- Любимый цвет

Ты должен задавать вопросы один за другим. Если ответ некорректный (например, возраст не число),
объясни это и попроси повторить. Когда все данные собраны, сохрани полученные данные'''
)

In [4]:
from langchain_core.messages import HumanMessage, AIMessage

profile_saved = False
# Создаем список для хранения истории сообщений
messages = [HumanMessage(content="Начнем процесс сбора информации о пользователе.")]

# Запускаем цикл взаимодействия с агентом
while not profile_saved:
    print(f'debug: before {profile_saved=}')
    # Запускаем агента с текущей историей сообщений
    response = agent.invoke({"messages": messages})
    
    
    # Добавляем ответ агента в историю
    messages.append(AIMessage(content=response["messages"][-1].content))
    
    # Выводим последний ответ агента
    print("\nАгент:", response["messages"][-1].content)
    print(f'after: {profile_saved=}')
    # Если профиль уже сохранен, выходим из цикла
    if profile_saved:
        break
        
    # Получаем ввод пользователя
    user_input = input("\nВаш ответ: ")
    if user_input == '_exit':
        break
    
    # Добавляем сообщение пользователя в историю
    messages.append(HumanMessage(content=user_input))

print("\nПроцесс сбора данных завершен!")

debug: before profile_saved=False

Агент: Привет! Я рад начать сбор информации о тебе. Чтобы начать, мне нужно знать твоё имя. Пожалуйста, введи своё имя. Я буду ждать твоего ответа, чтобы затем спросить о твоём возрасте. После этого, я также спрошу о твоём любимом жанре книг и любимом цвете. Это поможет мне лучше понять тебя и сохранить твои данные. Как тебя зовут?
after: profile_saved=False
debug: before profile_saved=False

Агент: Приятно познакомиться, Святослав! Теперь мне нужно знать твой возраст. Пожалуйста, введи свой возраст. Помни, что возраст должен быть целым числом больше нуля. Например, 25 или 30. Я буду ждать твоего ответа.
after: profile_saved=False
debug: before profile_saved=False

Агент: Извини, Святослав! Ты прав, возраст - это личная информация. Но в данном случае, я просто пытаюсь собрать некоторую базовую информацию о тебе, чтобы сохранить твой профиль. Если ты не хочешь делиться своим возрастом, ты можешь просто ввести "не скажу" или "не хочу говорить". Но если 

BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<function=save_profile>{"name": "Святослав", "age": 65, "genre": "фэнтези", "color": "желтый"}</function>'}}